# Notebook for building an XGBoost model for the Sendgrid sign up dataset
***
**Jake Mitchell Scott Schubert**

Initially using XGBoost

In [1]:
#To do anything
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

#Xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#random forrest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#One-hot encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


In [2]:
def one_hot_boi(col, expect_type):
    values = df[col]
    # integer encode
    label_encoder = LabelEncoder()
    values = label_encoder.fit_transform(values.astype(expect_type))
    integer_encoded = label_encoder.fit_transform(values)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded


In [3]:
train_path = 'data/signup_train_data.csv'
test_path = 'data/signup_test_data.csv'


# Load the data into a DataFrame 
df = pd.read_csv(train_path, low_memory=False)
test_df = pd.read_csv(test_path, low_memory=False)

# Split into X and Y
dfLabels = df.pop("label")
#print (df.head())

#Feature engineering
'''
    Giving up on one-hot-encoding for now.  Not converting to 
    a friendly data format for our pandas dataFrame and I'm not 
    convinced it will assist very much and would rather invest time
    into feature engineering specific columns
state_ohe = one_hot_boi('state', 'str')
aoeu = pd.DataFrame(state_ohe.T)
df['state_ohe'] = 0
count = 0
for row in state_ohe:
    aoeu = pd.DataFrame(row)
    df['state_ohe'].append(aoeu)
    print(aoeu)
'''

#name_notes
'''
df['name_notes_ohe'] = df['name_notes']
ohe = one_hot_boi('name_notes', 'str')
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000
ohe = pd.DataFrame(ohe)
print(ohe)
#Pursue.  "Invalid" with a "name_score" of 0 is really likely to be bad apple
#Want to ohe the options for name_notes....should only be like 7 of them
"First equals Last"
'''


#Feature Engineering

for dataframe in [df, test_df]:
    
    dataframe['employee_count_high'] = 0
    dataframe.loc[df['employee_count']  == '1 - 500', 'employee_count_high'] = 500
    dataframe.loc[df['employee_count']  == '501 - 1,000', 'employee_count_high'] = 1000
    dataframe.loc[df['employee_count']  == '1,001 - 5,000', 'employee_count_high'] = 5000
    dataframe.loc[df['employee_count']  == '5,000+', 'employee_count_high'] = 10000
    dataframe['volume_high'] = 0
    dataframe.loc[df['volume']  == '1 - 100,000', 'volume_high'] = 100000
    dataframe.loc[dataframe['volume']  == '100,001 - 2,500,000', 'volume_high'] = 2500000
    dataframe.loc[dataframe['volume']  == '2,500,001 - 10,000,000', 'volume_high'] = 10000000
    dataframe.loc[dataframe['volume']  == '10,000,000+', 'volume_high'] = 20000000
    dataframe.loc[dataframe['volume']  == '0 to 40k', 'volume_high'] = 40000
    dataframe.loc[dataframe['volume']  == '40k to 100k', 'volume_high'] = 100000
    dataframe['developer'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Developer', 'developer'] = 1
    dataframe['ceo'] = 0
    dataframe.loc[dataframe['user_persona']  == 'CEO', 'ceo'] = 1
    dataframe['other'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Other', 'other'] = 1
    dataframe['marketing'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Marketing', 'marketing'] = 1
    dataframe['technical_support'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Technical Support', 'technical_support'] = 1
    
    dataframe['name_notes_inv'] = 0
    dataframe.loc[dataframe['name_notes']  == 'Invalid', 'name_notes_inv'] = 1
    dataframe['name_notes_firstlast'] = 0
    dataframe.loc[dataframe['name_notes']  == 'First equals Last', 'name_notes_firstlast'] = 1
    dataframe['name_notes_inv_firstlast'] = 0
    dataframe.loc[dataframe['name_notes']  == 'Invalid|First equals Last', 'name_notes_inv_firstlast'] = 1
    dataframe['name_notes_sus_firstlast'] = 0
    dataframe.loc[dataframe['name_notes']  == 'Suspect|First equals Last', 'name_notes_sus_firstlast'] = 1
    dataframe['name_notes_sus'] = 0
    dataframe.loc[dataframe['name_notes']  == 'Suspect', 'name_notes_sus'] = 1
    dataframe['name_notes_inv_sus_firstlast'] = 0
    dataframe.loc[dataframe['name_notes']  == 'Invalid|Suspect|First equals Last', 'name_notes_inv_sus_firstlast'] = 1
    dataframe['name_notes_inv_sus'] = 0
    dataframe.loc[dataframe['name_notes']  == 'Invalid|Suspect', 'name_notes_inv_sus'] = 1
    dataframe['name_notes_tagbad'] = 0
    dataframe.loc[dataframe['name_notes']  == 'Tagged Bad', 'name_notes_tagbad'] = 1

    


#Temporarily parse non numerical data
frames_to_remove = ['zip', 'city', 'website', 'state', 'country', 'registration_ip', 'company','multifactor_country_code','created_at','lead_source','marketing_channel','volume','user_persona','initial_package','employee_count','geolocation_notes','name_notes','ip_notes','community_notes','email_notes','activity_notes','fingerprint_notes']
df = df.drop(frames_to_remove, axis = 1)
test_df = test_df.drop(frames_to_remove, axis = 1)

#print(df.head())
#print(df['name_notes_ohe'])


In [82]:
#XGBoost approach 

def xgb(seed_in, TO_TEST):
    # split data into train and test sets
    seed = seed_in
    test_size = 0.7
    X_train, X_test, y_train, y_test = train_test_split(df, dfLabels, test_size=test_size, random_state=seed)

    # fit model no training data
    model = XGBClassifier(depth = 9, eta = 0.15, n_estimators = 120)
    model.fit(X_train, y_train)
    
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("xgb - Seed: %i    -  Accuracy: %.2f%%" % (seed,accuracy * 100.0))
    
    if TO_TEST:
        #make predictions for train data
        y_pred = model.predict(test_df)
        predictions = [round(value) for value in y_pred]

        d = {'id':test_df['id'], 'label':predictions}
        preds = pd.DataFrame(data=d)

        preds.to_csv('predictions.csv', index=False)
        return model.predict_proba(test_df)
        
    return model.predict_proba(X_test)

In [83]:
#Random forrest approach 

def rf(seed, TO_TEST):
    seed = seed
    df2 = df
    df2 = df2.fillna(0)
    X_train, X_test, y_train, y_test = train_test_split(df2, dfLabels, test_size=0.7, random_state=seed)

    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=10, max_features='auto', max_leaf_nodes=None,
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=27, n_jobs=1,
                oob_score=False, random_state=0, verbose=0, warm_start=False)

    # Use the forest's predict method on the test data
    y_pred = clf.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("RF - Seed: %i    -  Accuracy: %.2f%%" % (seed,accuracy * 100.0))
    
    if TO_TEST:
        test_df2 = test_df
        test_df2 = test_df2.fillna(0)
        
        #make predictions for train data
        y_pred2 = clf.predict(test_df2)
        predictions = [round(value) for value in y_pred2]

        d = {'id':test_df['id'], 'label':predictions}
        preds = pd.DataFrame(data=d)

        preds.to_csv('predictions.csv', index=False)
        return clf.predict_proba(test_df2)
    
    return (clf.predict_proba(X_test), y_test)


In [87]:
def combine(seed, TO_TEST, rf_weight, xgb_weight):           
    if TO_TEST:
        x = rf(seed, TO_TEST)
    else:
        #Acquire training sample
        (x,y_test) = rf(seed, TO_TEST)
    rand_score = [b for a,b in x]
    
    xgb_score = xgb(seed, TO_TEST)
    xgb_score = [b for a,b in xgb_score]   

    predictions = []
    for i in range(0,len(rand_score)):
        val = rand_score[i]*rf_weight + xgb_score[i]*xgb_weight
        predictions.append(val)
        
    predictions = [int(round(value)) for value in predictions]
    
    if TO_TEST:
        d = {'id':test_df['id'], 'label':predictions}
        preds = pd.DataFrame(data=d)
        preds.to_csv('predictions.csv', index=False)
        
    else:
        accuracy = accuracy_score(y_test, predictions)
        print("Combine - Seed: %i    -  Accuracy: %.2f%%" % (seed,accuracy * 100.0))       
    
    
#seed, TO_TEST, rand forest weighting, xgboost weighting
combine(1, True, 0.3, 0.7)



RF - Seed: 1    -  Accuracy: 93.48%


/Users/schubydooo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


xgb - Seed: 1    -  Accuracy: 96.07%


/Users/schubydooo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
Equal weighting: 95.74
0.3/0.7 - 95.98


In [90]:
def combine_findWeight(seed):           
    #Acquire training sample
    (x,y_test) = rf(seed, False)
    rand_score = [b for a,b in x]
    
    xgb_score = xgb(seed, False)
    xgb_score = [b for a,b in xgb_score]   

    print("SEED ------------ %i" % (seed))
    for rf_weight in range(1,100):
        rf_weight = float(rf_weight/100)
        xgb_weight = 1-rf_weight
        print(rf_weight)
        print(xgb_weight)
        predictions = []
        for i in range(0,len(rand_score)):
            val = rand_score[i]*rf_weight + xgb_score[i]*xgb_weight
            predictions.append(val)
        predictions = [int(round(value)) for value in predictions]
        accuracy = accuracy_score(y_test, predictions)
        print("rf . xgb = %f . %f  -  Accuracy: %.2f%%" % (rf_weight, xgb_weight, accuracy * 100.0))  
        print()

    
#seed, TO_TEST, rand forest weighting, xgboost weighting
combine_findWeight(1)
#for i in range(0,20):
    #combine_findWeight(i)



RF - Seed: 1    -  Accuracy: 93.48%


/Users/schubydooo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


xgb - Seed: 1    -  Accuracy: 96.07%
SEED ------------ 1
0.01
0.99

rf . xgb = 0.010000 . 0.990000  -  Accuracy: 96.07%
0.02
0.98

rf . xgb = 0.020000 . 0.980000  -  Accuracy: 96.07%
0.03
0.97

rf . xgb = 0.030000 . 0.970000  -  Accuracy: 96.06%
0.04
0.96

rf . xgb = 0.040000 . 0.960000  -  Accuracy: 96.07%
0.05
0.95

rf . xgb = 0.050000 . 0.950000  -  Accuracy: 96.06%
0.06
0.94

rf . xgb = 0.060000 . 0.940000  -  Accuracy: 96.06%
0.07
0.9299999999999999

rf . xgb = 0.070000 . 0.930000  -  Accuracy: 96.06%
0.08
0.92

rf . xgb = 0.080000 . 0.920000  -  Accuracy: 96.05%
0.09
0.91

rf . xgb = 0.090000 . 0.910000  -  Accuracy: 96.05%
0.1
0.9

rf . xgb = 0.100000 . 0.900000  -  Accuracy: 96.05%
0.11
0.89

rf . xgb = 0.110000 . 0.890000  -  Accuracy: 96.06%
0.12
0.88

rf . xgb = 0.120000 . 0.880000  -  Accuracy: 96.06%
0.13
0.87

rf . xgb = 0.130000 . 0.870000  -  Accuracy: 96.06%
0.14
0.86

rf . xgb = 0.140000 . 0.860000  -  Accuracy: 96.06%
0.15
0.85

rf . xgb = 0.150000 . 0.850000  -  Acc

In [80]:
#xgb(31,False)

#for x in range (1,10):
#    xgb(x, False)


rf(1, True)
#xgb(1, False)

#for x in range (1,10):
#    rf(x, False)

RF - Seed: 1    -  Accuracy: 90.60%


array([[0.8510252 , 0.1489748 ],
       [0.8510252 , 0.1489748 ],
       [0.8510252 , 0.1489748 ],
       ...,
       [0.67682674, 0.32317326],
       [0.2146204 , 0.7853796 ],
       [0.8510252 , 0.1489748 ]])

In [31]:
rf(2, False)
xgb(2, False)       #just all

RF - Seed: 2    -  Accuracy: 93.68%


/Users/schubydooo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


xgb - Seed: 2    -  Accuracy: 95.93%


In [26]:
rf(1, False)
xgb(1, False)   #baseline

RF - Seed: 1    -  Accuracy: 94.46%
xgb - Seed: 1    -  Accuracy: 96.03%


/Users/schubydooo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
